https://deepmlblog.wordpress.com/2016/01/05/residual-networks-in-torch-mnist/

In [42]:
require 'torch'
require 'nn'
require 'optim'
require 'image'
torch.manualSeed(1)

function reload(name)
    package.loaded[name] = nil
    return require(name)
end
utils = reload('./utils')
models = reload('./models')
datasets = reload('./datasets')
Logger = reload('./logger')


In [41]:

train_data, train_label,
    validation_data, validation_label = utils.load_data()

train_data = train_data[{{1,1000}}]
train_label = train_label[{{1, 1000}}]


In [135]:
print('train')
print(train_data:mean())
print(train_data:std())
print(train_data:min())
print(train_data:max())
print('validation')
print(validation_data:mean())
print(validation_data:std())
print(validation_data:min())
print(validation_data:max())


train	
-0.0078593129846774	


0.99036449947161	


-0.42407388981865	
2.8215433152638	
validation	


-0.0048816247909239	


0.99368015338313	


-0.42407388981865	


2.8215433152638	


In [42]:

SEED = 1
MODEL_NAME = 'mlp_3'
TITLE = 'mnist_' .. MODEL_NAME
-- VARIANT = 1
-- LOGPATH = './results/' .. TITLE .. '/' .. VARIANT
EPOCHS = 10

p1 = {
   learningRate = 1e-1,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p2 = {
   learningRate = 1e-1,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p3 = {
   learningRate = 1e-2,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p4 = {
   learningRate = 1e-2,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p5 = {
   learningRate = 1e-3,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p6 = {
   learningRate = 1e-3,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p7 = {
   learningRate = 1e-4,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p8 = {
   learningRate = 1e-4,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

sgd_params = {p1, p2, p3, p4, p5, p6, p7, p8}

batch_params = {
    n_of_batches = 100
}

-- for i=1,1 do
for i=1,#sgd_params do
    VARIANT = i
    LOGPATH = './results/' .. TITLE .. '/' .. VARIANT
    start_training(sgd_params[i], batch_params)
end


In [30]:

function start_training(sgd_params, batch_params)
    
    torch.manualSeed(SEED)
    
    local n_of_batches = batch_params.n_of_batches
    
    model = models[MODEL_NAME]()
    criterion = nn.ClassNLLCriterion()

    x, dl_params = model:getParameters()

    info_log = Logger(
        LOGPATH, 
        {
            general_info = {
                title = TITLE,
                description = '',
                seed = SEED,
                model = tostring(model),
                criterion = tostring(criterion),
            },
            data_size = {
                train_data = torch.totable(train_data:size()),
                train_label = torch.totable(train_label:size()),
                validation_data = torch.totable(validation_data:size()),
                validation_label = torch.totable(validation_label:size()),
            },
            sgd_params = sgd_params
        })
    info_log:write()

    logger = optim.Logger(LOGPATH .. '/accuracy.log')
    logger:setNames{'train_loss', 'validation_loss'}

    time_start = os.time()
    
    batch_size = train_data:size()[1] / n_of_batches
    
    for epoch=1,EPOCHS do
        
        assert(train_data:mean() > -0.1 and train_data:mean() < 0.1)
        assert(validation_data:mean() > -0.1 and validation_data:mean() < 0.1)

        train_loss = 0
        
        for j=1,n_of_batches do
            batch_data = utils.get_batch(train_data, j, batch_size)
            batch_label = utils.get_batch(train_label, j, batch_size)

            sys.tic()
            _, fs = optim.sgd(utils.feval,x,sgd_params)
            duration = sys.toc()
            train_loss = train_loss + fs[1]
        end
        
        train_loss = train_loss/n_of_batches

        validation_loss = calc_validation_loss(model, criterion, validation_data, validation_label)
        logger:add{train_loss, validation_loss}

    end

    total_train_time = os.time() - time_start
    
    train_accuracy = utils.calc_accuracy(model, train_data, train_label)
    validation_accuracy = utils.calc_accuracy(model, validation_data, validation_label)
    
    info_log:write_result({
            model_name = MODEL_NAME,
            variant = VARIANT,
            epochs = EPOCHS,
            learningRate = sgd_params.learningRate,
            weightDecay = sgd_params.weightDecay,
            total_train_time = total_train_time,
            train_loss = train_loss,
            validation_loss = validation_loss,
            train_accuracy = train_accuracy,
            validation_accuracy = validation_accuracy,
        })
--     info_log:write_result(
--         total_train_time, train_loss, validation_loss, train_accuracy, validation_accuracy
--     )

    logger:style{'+-', '+-'}
    logger:plot()
end


In [44]:
-- infos = json.load('./experiments/experiment_1/structures.json')
-- -- print(infos[1])

-- info = infos[1]

function load_structure(info)

    model_name = info.model
    data_size = info.data_size
    dataset_name = info.dataset
    criterion_name = info.criterion
    optim_name = info.optim

    train_data, train_label, validation_data, validation_label =  datasets[dataset_name]()

    if data_size ~= 0 then
        train_data = train_data[{{1,data_size}}]
        train_label = train_label[{{1,data_size}}]
    end

    model = models[model_name]()
    criterion = nn[criterion_name]()
    optimization = optim[optim_name]
    dataset = {train_data, train_label, validation_data, validation_label}

    return {
        model = model,
        criterion = criterion,
        optimization = optimization,
        dataset = dataset,
    }

end

-- structure = load_from_infos(info)

-- variants = json.load('./experiments/experiment_1/variants.json')
-- variant = variants[1]

function train(structure_json, variant)
    
    local structure = load_structure(structure_json)    
    local model = structure.model
    local criterion = structure.criterion
    local optimization = structure.optimization
    local dataset = structure.dataset
    local train_data, train_label, validation_data, validation_label =
        dataset[1], dataset[2], dataset[3], dataset[4]
    
    local sgd_params = variant.optim
    local train_params = variant.train

    torch.manualSeed(1)
    local batch_size = train_params.batch_size
    batch_size = batch_size == 0 and train_data:size()[1] or batch_size
    local n_of_batches = train_data:size()[1] / batch_size

    local epochs = 2
    
    logger = Logger(
        './results_2/' .. 'experiment_1', --path
        {'train_loss', 'validation_loss'}, -- log headers
        {
            structure = structure_json,
            variant = variant,
        }
    )

    x, dl_params = model:getParameters()
    local time_start = os.time()

    for epoch=1,epochs do

        local train_loss = 0
        
        for batch_number=1,n_of_batches do
            batch_data = utils.get_batch(train_data, batch_number, batch_size)
            batch_label = utils.get_batch(train_label, batch_number, batch_size)

--             sys.tic()
            _, fs = optimization(utils.feval,x,sgd_params)
--             duration = sys.toc()
            train_loss = train_loss + fs[1]

        end
                
        train_loss = train_loss/n_of_batches
        
        local validation_loss = calc_validation_loss(model, criterion, validation_data, validation_label)

        logger:append_log({train_loss, validation_loss})
        
        local train_accuracy = utils.calc_accuracy(model, train_data, train_label)
        local validation_accuracy = utils.calc_accuracy(model, validation_data, validation_label)
        local total_train_time = os.time() - time_start
        
        logger:write_results({
                train_accuracy = train_accuracy,
                validation_accuracy = validation_accuracy,
                total_train_time = total_train_time,
            })
        
        print('train_accuracy', train_accuracy)
        print('validation_accuracy', validation_accuracy)

    end
end

-- train(structure, variant)

structures_json = json.load('./experiments/experiment_1/structures.json')
variants_json = json.load('./experiments/experiment_1/variants.json')

train(structures_json[1], variants_json[1])
-- train(, './experiments/experiment_1/variants.json')




train_accuracy	0.702	
validation_accuracy	0.6501	


train_accuracy	0.88	
validation_accuracy	0.8232	


In [18]:

function start_training(sgd_params, batch_params)
    
    torch.manualSeed(SEED)
    
    local n_of_batches = batch_params.n_of_batches
    
    model = models[MODEL_NAME]()
    criterion = nn.ClassNLLCriterion()

    x, dl_params = model:getParameters()

    info_log = Logger(
        LOGPATH,
        {
            general_info = {
                title = TITLE,
                description = '',
                seed = SEED,
                model = tostring(model),
                criterion = tostring(criterion),
            },
            data_size = {
                train_data = torch.totable(train_data:size()),
                train_label = torch.totable(train_label:size()),
                validation_data = torch.totable(validation_data:size()),
                validation_label = torch.totable(validation_label:size()),
            },
            sgd_params = sgd_params
        })
    info_log:write()

    logger = optim.Logger(LOGPATH .. '/accuracy.log')
    logger:setNames{'train_loss', 'validation_loss'}

    time_start = os.time()
    
    batch_size = train_data:size()[1] / n_of_batches
    
    for epoch=1,EPOCHS do
        
        assert(train_data:mean() > -0.1 and train_data:mean() < 0.1)
        assert(validation_data:mean() > -0.1 and validation_data:mean() < 0.1)

        train_loss = 0
        
        for j=1,n_of_batches do
            batch_data = utils.get_batch(train_data, j, batch_size)
            batch_label = utils.get_batch(train_label, j, batch_size)

            sys.tic()
            _, fs = optim.sgd(utils.feval,x,sgd_params)
            duration = sys.toc()
            train_loss = train_loss + fs[1]
        end
        
        train_loss = train_loss/n_of_batches

        validation_loss = calc_validation_loss(model, criterion, validation_data, validation_label)
        logger:add{train_loss, validation_loss}

    end

    total_train_time = os.time() - time_start
    
    train_accuracy = utils.calc_accuracy(model, train_data, train_label)
    validation_accuracy = utils.calc_accuracy(model, validation_data, validation_label)
    
    info_log:write_result({
            model_name = MODEL_NAME,
            variant = VARIANT,
            epochs = EPOCHS,
            learningRate = sgd_params.learningRate,
            weightDecay = sgd_params.weightDecay,
            total_train_time = total_train_time,
            train_loss = train_loss,
            validation_loss = validation_loss,
            train_accuracy = train_accuracy,
            validation_accuracy = validation_accuracy,
        })
--     info_log:write_result(
--         total_train_time, train_loss, validation_loss, train_accuracy, validation_accuracy
--     )

    logger:style{'+-', '+-'}
    logger:plot()
end

1	2	3	4	


In [55]:
dofile('./run_experiment.lua')

train_accuracy	0.702	
validation_accuracy	0.6501	


train_accuracy	0.88	
validation_accuracy	0.8232	


train_accuracy	0.906	
validation_accuracy	0.8418	


train_accuracy	0.919	
validation_accuracy	0.847	


train_accuracy	0.93	
validation_accuracy	0.8515	


train_accuracy	0.938	
validation_accuracy	0.8547	


train_accuracy	0.943	
validation_accuracy	0.8588	


train_accuracy	0.949	
validation_accuracy	0.8608	


train_accuracy	0.953	
validation_accuracy	0.8628	


train_accuracy	0.958	
validation_accuracy	0.8638	



In [67]:

require 'pl'

tbl_1 = {
    aaa = 'aaa',
    b = 'bbb'
}

tbl_2 = {
    c = 'ccc',
    d = 'ddd'
}

tbl_3 = {
    eee = 'eee',
    zzz = 'zzz'
}

tbl_4 = {
    qqq = 'qqq',
    aaa = 'zzz'
}

function merge_tables(tables)
  tbl = {}
  for i=1,#tables do
    tbl = tablex.merge(tbl, tables[i], true)
  end
  return tbl
end

res = merge_tables({tbl_1, tbl_2, tbl_3, tbl_4})
-- res = tablex.merge(tbl_1, tbl_2, true)

print(res)
-- print(res)

4	
{eee="eee",aaa="zzz",zzz="zzz",qqq="qqq",d="ddd",c="ccc",b="bbb",i="sdf"}
